In [3]:
import os
import pandas as pd
import unidecode
from datetime import time
import numpy as np


# PROCESSING ET CLEANING ✅

In [4]:
# Charger les fichiers CSV
def load_csv_files(csv_folder, file_prefix="EXP_", sep=';'):
    dataframes = {}
    files = [f for f in os.listdir(csv_folder) if f.startswith(file_prefix) and f.endswith(".csv")]
    for file in files:
        df_name = file.replace(file_prefix, "").replace(".csv", "").capitalize()
        df = pd.read_csv(os.path.join(csv_folder, file), sep=sep)
        dataframes[df_name] = df
    return dataframes

# Sélection et renommage des colonnes
def select_and_rename_columns(dataframes, config):
    for name, settings in config.items():
        if name in dataframes:
            df = dataframes[name]
            if 'keep' in settings:
                df = df[settings['keep']]
            if 'rename' in settings:
                df.rename(columns=settings['rename'], inplace=True)
            if 'drop_pattern' in settings:
                drop_cols = df.filter(regex=settings['drop_pattern']).columns
                df.drop(columns=drop_cols, inplace=True)
            dataframes[name] = df

# Nettoyage des noms de colonnes
def clean_column_names(dataframes):
    for name, df in dataframes.items():
        df.columns = [unidecode.unidecode(col).lower().replace(" ", "_").replace("-", "_") for col in df.columns]
        dataframes[name] = df

# Conversion des types avec gestion spécifique pour la colonne `sur`
def convert_column_types(dataframes, column_types):
    for name, df in dataframes.items():
        for col in df.columns:
            # Conversion des colonnes string
            if col in column_types.get('string', []):
                df[col] = df[col].astype(str)

            # Conversion spécifique pour la colonne `sur`
            elif col == 'sur':
                # Extraire la valeur après '/' et convertir en entier
                df[col] = pd.to_numeric(df[col].astype(str).str.replace('/', ''), errors='coerce').astype('Int64')

            # Conversion des colonnes float
            elif col in column_types.get('float', []):
                df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

            # Conversion des colonnes int
            elif col in column_types.get('int', []) and col != 'sur':  # Exclure `sur` pour éviter un double traitement
                # Convertir d'abord en float pour traiter les valeurs décimales et les `NaN`
                temp_col = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

                # Vérifier si toutes les valeurs non `NaN` sont des entiers
                if (temp_col.dropna() % 1 == 0).all():
                    # Convertir en `Int64` en utilisant `.fillna(np.nan).astype('Int64')`
                    df[col] = temp_col.fillna(np.nan).astype('Int64')
                else:
                    # Si des valeurs décimales existent, laisser en `float`
                    df[col] = temp_col

            # Conversion des colonnes date
            elif col in column_types.get('date', []):
                df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)

            # Conversion des colonnes time
            elif col in column_types.get('time', []):
                df[col] = pd.to_datetime(df[col].astype(str).str.replace('h', ':'), format='%H:%M', errors='coerce').dt.time

            # Conversion des colonnes category
            elif col in column_types.get('category', []):
                df[col] = df[col].astype('category')

        # Mettre à jour le DataFrame dans le dictionnaire
        dataframes[name] = df

def anonymize_students(dataframes, eleve_df, ident_column='ident'):
    """
    Anonymise les informations des élèves dans les DataFrames spécifiés.

    Parameters:
        dataframes (dict): Dictionnaire de DataFrames à anonymiser.
        eleve_df (DataFrame): DataFrame contenant les informations des élèves avec l'identifiant.
        ident_column (str): Nom de la colonne d'identifiant dans eleve_df (par défaut 'ident').
    """
    # Créer le mapping pour les élèves
    eleve_mapping = {f"{row['nom']} {row['prenom']}": row[ident_column] for _, row in eleve_df.iterrows()}

    # Liste des DataFrames à anonymiser pour les élèves
    student_dfs = ['Punition', 'Notes','Notesdevoir_t1','Notesdevoir_t2', 'Notesdevoir_t3', 'Absenceseleves', 'Retards', 'Appreciationprofesseurs',
                   'Passagesinfirmerie', 'Appreciationduconseil', 'Sanction', 'Notes','Absencerepas']

    for name in student_dfs:
        if name in dataframes:
            df = dataframes[name]
            if 'nom' in df.columns and 'prenom' in df.columns:
                df['nom_prenom'] = df['nom'] + ' ' + df['prenom']
                df['eleve_id'] = df['nom_prenom'].map(eleve_mapping)
                df.drop(columns=['nom', 'prenom', 'nom_prenom'], inplace=True)
                dataframes[name] = df

def anonymize_professors(dataframes, professor_df, ident_column='ident'):
    """
    Anonymise les informations des professeurs dans les DataFrames spécifiés.

    Parameters:
        dataframes (dict): Dictionnaire de DataFrames à anonymiser.
        professor_df (DataFrame): DataFrame contenant les informations des professeurs avec l'identifiant.
        ident_column (str): Nom de la colonne d'identifiant dans professor_df (par défaut 'ident').
    """
    # Créer le mapping pour les professeurs
    professor_df['nom_complet'] = professor_df['nom'] + ' ' + professor_df['prenom']
    prof_mapping = {row['nom_complet']: row[ident_column] for _, row in professor_df.iterrows()}

    # Liste des DataFrames à anonymiser pour les professeurs
    professor_dfs = ['Notesdevoir_t1','Notesdevoir_t2', 'Notesdevoir_t3', 'Appreciationprofesseurs', 'Service', 'Notes']

    for name in professor_dfs:
        if name in dataframes and 'profs' in dataframes[name].columns:
            df = dataframes[name]
            df['profs_standard'] = df['profs'].str.replace(r'^M\.|Mme\s+', '', regex=True).str.strip()
            df['prof_id'] = df['profs_standard'].map(prof_mapping)
            df.drop(columns=['profs', 'profs_standard'], inplace=True)
            dataframes[name] = df


In [5]:
# Configurer la sélection et le renommage des colonnes
columns_config = {
    'Punition': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'PUNITION', 'DATE', 'MOTIF', 'HEURE']
    },
    'Notesdevoir_t1': {
        'keep': ['CLASSES', 'NOM', 'PRENOM', 'PROFS', 'MATIERE', 'DATE', 'P1', 'SUR', 'COEFF', 'NOTE'],
        'rename': {'P1': 'TRIMESTRE'}

    },
    'Notesdevoir_t2': {
        'keep': ['CLASSES', 'NOM', 'PRENOM', 'PROFS', 'MATIERE', 'DATE', 'P1', 'SUR', 'COEFF', 'NOTE'],
        'rename': {'P1': 'TRIMESTRE'}

    },
    'Notesdevoir_t3': {
        'keep': ['CLASSES', 'NOM', 'PRENOM', 'PROFS', 'MATIERE', 'DATE', 'P1', 'SUR', 'COEFF', 'NOTE'],
        'rename': {'P1': 'TRIMESTRE'}

    },

    'Professeur': {
        'keep': ['IDENT', 'CIVILITE', 'NOM', 'PRENOM']
    },
    'Absenceseleves': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'MOTIF', 'DATE DEBUT', 'DEMI JOUR'],
        'rename': {'DATE DEBUT': 'DATE', 'DEMI JOUR': 'NB_DEMI_JOURNEE'}
    },
    'Absencerepas': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'DATE']
    },
    'Retards': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'MOTIF', 'REGLE', 'DATE', 'HEURE', 'DUREE']
    },
    'Eleve': {
        'keep': ['IDENT', 'NOM', 'PRENOM', 'DATE NAISS', 'CLASSES', 'SEXE'],
        'rename': {'DATE NAISS': 'DATE_NAISS'}
    },
    'Service': {
        'keep': ['MATIERE', 'CLASSES', 'PROFS', 'NB DEVOIRS', 'COEFF'],
        'rename': {'NB DEVOIRS': 'NB_DEVOIRS'}
    },
    'Appreciationprofesseurs': {
        'keep': ['NOM', 'PRENOM', 'MATIERE', 'CLASSE/GRPE', 'PROFS', 'PERIODE', 'APPR.A'],
        'rename': {'CLASSE/GRPE': 'CLASSE', 'APPR.A': 'APPR'}
    },
    'Passagesinfirmerie': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'H DEBUT', 'DATE', 'DUREE'],
        'rename': {'H DEBUT': 'HEURE'}
    },
    'Appreciationduconseil': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'PERIODE', 'APPR.C'],
        'rename': {'APPR.C': 'APPR'}
    },
    'Sanction': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'DATE', 'SANCTION', 'MOTIF']
    },
    'Notes': {
        'keep': ['PROFS', 'MATIERE', 'CLASSE/GRPE', 'NOM ELEVE', 'PRENOM ELEVE', 'MOY_ELEVE'] +
                [f'Devoir{i} - Note' for i in range(1, 25)] +
                [f'Devoir{i} - Coeff' for i in range(1, 25)] +
                [f'Devoir{i} - Date' for i in range(1, 25)] +
                [f'Devoir{i} - Période Notation' for i in range(1, 25)],
                'rename': {'NOM ELEVE': 'NOM', 'PRENOM ELEVE':'PRENOM'},
        'drop_pattern': 'Devoir\d+ - Facultatif'
    }
}

# Configurer les types de colonnes
column_types = {
    'string': ['nom', 'prenom', 'classes','profs', 'civilite', 'motif', 'punition', 'regle'],
    'float': ['note', 'moyenne'],
    'int': ['nb_demi_journee', 'nb_devoirs','coeff', 'sur'],
    'date': ['date', 'date_naiss', 'periode'],
    'time': ['heure', 'duree'],
    'category': ['classe', 'trimestre', 'matiere', 'sexe']
}


# Application
csv_folder_path = "../../../raw_data/Daudet/2023-2024/Pronote/"

#extraction des données
data_pronote = load_csv_files(csv_folder=csv_folder_path)

#selection des colonnes
select_and_rename_columns(data_pronote, columns_config)

#cleaning des noms de colonnes
clean_column_names(data_pronote)

#cleaning des types de colonnes
convert_column_types(data_pronote, column_types)

# Appliquer l'anonymisation des élèves
anonymize_students(data_pronote, data_pronote['Eleve'])

# Appliquer l'anonymisation des professeurs
anonymize_professors(data_pronote, data_pronote['Professeur'])

/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_78255/2906436130.py:7: DtypeWarning: Columns (81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(csv_folder, file), sep=sep)
/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_78255/2906436130.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=settings['rename'], inplace=True)
/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_78255/2906436130.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

#### Tous les dataframes sont crées, structurés, nettoyées, anonymisés ✅

# DF_NOTES

In [6]:
# Ajouter une colonne 'trimestre' pour chaque DataFrame avant de les concaténer
data_pronote['Notesdevoir_t1']['trimestre'] = 'Trimestre 1'
data_pronote['Notesdevoir_t2']['trimestre'] = 'Trimestre 2'
data_pronote['Notesdevoir_t3']['trimestre'] = 'Trimestre 3'

# Concaténer les trois DataFrames
df_notes_devoir = pd.concat([
    data_pronote['Notesdevoir_t1'],
    data_pronote['Notesdevoir_t2'],
    data_pronote['Notesdevoir_t3']
], ignore_index=True)

# Afficher un aperçu du DataFrame concaténé
df_notes_devoir.head()

,classes,matiere,date,trimestre,sur,coeff,note,eleve_id,prof_id
0,1A,FRANÇAIS,2023-09-21,Trimestre 1,20,1.0,13.0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,0-qkoCBCeu1-A2WvQZEecuK98nvosoTdzUTdiI6lwDo
1,1A,ENSEIGNEMENT SCIENTIFIQUE (PC),2023-09-25,Trimestre 1,20,0.5,7.0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,zalvxQljlC_iyENJQK4qClMhXqp3ZogCFPq9JR9Yalw
2,1A,SPE MATHS,2023-09-26,Trimestre 1,20,1.0,8.0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,JumFXm1J2Jm1zAFbJ-KKVcMFxl2mfiu882gBggAozMg
3,1A,HISTOIRE GEOGRAPHIE,2023-09-27,Trimestre 1,10,1.0,5.0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,MkTDEy0fzub_lzFNRBxLLG0sLd5mzSpMxX3wx8l6Guo
4,1A,ARABE LVA,2023-10-04,Trimestre 1,20,2.0,12.0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,t8jRAdLt_P0Ai2D2pyBamlLBFMnN_k9Od6ycTUB98oA


In [22]:
df_notes_devoir.sort_values(by='note',ascending=True).reset_index(drop=True)


,classes,matiere,date,trimestre,sur,coeff,note,eleve_id,prof_id
0,1B/C,SPE HGGSP,2024-02-12,Trimestre 2,5,1.0,0.0,xkfZC8M7V2xck6tJFMrDiyp_87Mm80smQwhClbFJdsQ,MkTDEy0fzub_lzFNRBxLLG0sLd5mzSpMxX3wx8l6Guo
1,2B,MATHEMATIQUES,2024-03-20,Trimestre 2,10,1.0,0.0,QoHj7t0BrPEr5cfr_rnnpjKm1k0HTw1JVHiiXB0okfw,PTjbz9bkuijqvOpCGAMANH4KbbF3p9S2aB6O_9x6Ttg
2,Terminale,HISTOIRE GEOGRAPHIE EN ARABE,2024-05-16,Trimestre 3,20,1.0,0.0,ZPT1dqFBSkQZ6qGy4Dtfa4uEaQNgXw2Cbu-45QI5ur0,Vds9QuvCmVJaZgret-n885B4AoMFQn8YAtZ9MTaVES4
3,1B/C,FRANÇAIS,2024-04-23,Trimestre 3,20,0.5,0.0,YswxOZrqTGLTW_jkdfurfq6pwpppKbTzfyWlvQoHqlA,o-CqBI-hQySHQ_mc2QMhTTATJcSs8FvHu2-TwcWKGsg
4,1B/C,FRANÇAIS,2024-04-23,Trimestre 3,20,0.5,0.0,QMSXYlPZubea3EiBa86bwVxIX9kQj0zX1XSATau4jiM,o-CqBI-hQySHQ_mc2QMhTTATJcSs8FvHu2-TwcWKGsg
...,...,...,...,...,...,...,...,...,...
23142,Terminale,HISTOIRE GEOGRAPHIE,2024-04-04,Trimestre 3,20,1.0,NaN,6Pbd7oxjZdFLB3wZNNmsG8gkSP2XhH23oNBDBLA5dE8,NaTUeIaRZA9cPVkJaVYXHesRe7jqnQ2GpN6t1ycqJz8
23143,Terminale,HISTOIRE GEOGRAPHIE,2024-04-04,Trimestre 3,20,1.0,NaN,oSHXzw2GIi4XBcygPsIKugb7S7aIo1ri4XO2UP_bRXo,NaTUeIaRZA9cPVkJaVYXHesRe7jqnQ2GpN6t1ycqJz8
23144,Terminale,SPE SES,2024-05-13,Trimestre 3,20,1.0,NaN,oSHXzw2GIi4XBcygPsIKugb7S7aIo1ri4XO2UP_bRXo,h9TBuMdPs82YwWigXSwir30nkmktZt7bfCvTtCxtKpo
23145,Terminale,SPE HGGSP,2024-05-20,Trimestre 3,20,2.0,NaN,oSHXzw2GIi4XBcygPsIKugb7S7aIo1ri4XO2UP_bRXo,NaTUeIaRZA9cPVkJaVYXHesRe7jqnQ2GpN6t1ycqJz8


# Creation du df_notes a partit des notes

In [8]:
data_pronote['Notesdevoir_t2']


,classes,matiere,date,trimestre,sur,coeff,note,eleve_id,prof_id
0,1A,SPE PC,2023-12-18,Trimestre 2,10,1.0,4.5,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,aR5r7eJWG3uAF4iOt4zShmtqKpBy0liTVOxdOWKG6_c
1,1A,SPE MATHS,2023-12-18,Trimestre 2,50,1.0,28.0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,JumFXm1J2Jm1zAFbJ-KKVcMFxl2mfiu882gBggAozMg
2,1A,HISTOIRE GEOGRAPHIE,2024-01-05,Trimestre 2,20,1.0,11.0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,MkTDEy0fzub_lzFNRBxLLG0sLd5mzSpMxX3wx8l6Guo
3,1A,HISTOIRE GEOGRAPHIE,2024-01-16,Trimestre 2,20,2.0,7.0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,MkTDEy0fzub_lzFNRBxLLG0sLd5mzSpMxX3wx8l6Guo
4,1A,FRANÇAIS,2024-01-17,Trimestre 2,20,1.0,8.0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,0-qkoCBCeu1-A2WvQZEecuK98nvosoTdzUTdiI6lwDo
...,...,...,...,...,...,...,...,...,...
7865,Terminale,HISTOIRE GEOGRAPHIE EN ARABE,2024-03-12,Trimestre 2,20,1.0,16.0,D-H5a7Qp9mw-1fNMLUSnF6gBBuMEndddzmZ7asqOU10,Vds9QuvCmVJaZgret-n885B4AoMFQn8YAtZ9MTaVES4
7866,Terminale,MATHS COMPLEMENTAIRES,2024-03-13,Trimestre 2,20,3.0,14.0,D-H5a7Qp9mw-1fNMLUSnF6gBBuMEndddzmZ7asqOU10,PTjbz9bkuijqvOpCGAMANH4KbbF3p9S2aB6O_9x6Ttg
7867,Terminale,HISTOIRE GEOGRAPHIE EN ARABE,2024-03-13,Trimestre 2,20,1.0,18.0,D-H5a7Qp9mw-1fNMLUSnF6gBBuMEndddzmZ7asqOU10,Vds9QuvCmVJaZgret-n885B4AoMFQn8YAtZ9MTaVES4
7868,Terminale,ARABE LV SECTION,2024-03-15,Trimestre 2,20,1.0,17.0,D-H5a7Qp9mw-1fNMLUSnF6gBBuMEndddzmZ7asqOU10,GYys8Q_Y-fzpN5CyZM9xMI8q4e5AyQJsejYIgSPHkQE


In [9]:
import pandas as pd

# Dates de référence pour chaque trimestre
trimestre_1_start, trimestre_1_end = pd.to_datetime("2023-08-28"), pd.to_datetime("2023-11-30")
trimestre_2_start, trimestre_2_end = pd.to_datetime("2023-12-01"), pd.to_datetime("2024-03-29")
trimestre_3_start, trimestre_3_end = pd.to_datetime("2024-03-30"), pd.to_datetime("2024-08-07")

def assign_trimestre(row, date_col):
    """
    Fonction pour attribuer un trimestre en fonction de la date.
    """
    if pd.notna(row[date_col]):
        if trimestre_1_start <= row[date_col] <= trimestre_1_end:
            return 'Trimestre 1'
        elif trimestre_2_start <= row[date_col] <= trimestre_2_end:
            return 'Trimestre 2'
        elif trimestre_3_start <= row[date_col] <= trimestre_3_end:
            return 'Trimestre 3'
    return None

# Appliquer la règle pour chaque colonne devoir{i}___periode_notation
for i in range(1, 25):
    date_col = f'devoir{i}___date'
    periode_col = f'devoir{i}___periode_notation'

    # Convertir la colonne de date en format datetime si ce n'est pas déjà le cas
    data_pronote['Notes'][date_col] = pd.to_datetime(data_pronote['Notes'][date_col], errors='coerce')

    # Appliquer la fonction assign_trimestre pour attribuer les trimestres
    data_pronote['Notes'][periode_col] = data_pronote['Notes'].apply(assign_trimestre, date_col=date_col, axis=1)


/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/projets_osui/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '29/01/2024' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/projets_osui/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '23/11/2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/projets_osui/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '25/09/2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/mat

In [10]:
# import pandas as pd

# # Initialiser une liste pour stocker les lignes du nouveau DataFrame
# transformed_data = []

# # Boucler sur chaque devoir (devoir1 à devoir24)
# for i in range(1, 25):
#     # Définir les noms de colonnes pour chaque devoir
#     note_col = f'devoir{i}___note'
#     coeff_col = f'devoir{i}___coeff'
#     date_col = f'devoir{i}___date'

#     # Filtrer les lignes où il y a une note pour le devoir i
#     devoir_data = data_pronote['Notes'][['eleve_id', 'matiere', 'prof_id', note_col, coeff_col, date_col]]

#     # Renommer les colonnes pour un format générique
#     devoir_data = devoir_data.rename(columns={
#         note_col: 'note',
#         coeff_col: 'coef',
#         date_col: 'date'
#     })

#     # Supprimer les lignes sans note pour le devoir i
#     devoir_data = devoir_data.dropna(subset=['note', 'coef', 'date'])

#     # Ajouter la colonne 'trimestre' en fonction des dates
#     devoir_data['trimestre'] = pd.cut(
#         pd.to_datetime(devoir_data['date']),
#         bins=[
#             pd.Timestamp("2023-08-28"),
#             pd.Timestamp("2023-11-30"),
#             pd.Timestamp("2024-03-29"),
#             pd.Timestamp("2024-08-07")
#         ],
#         labels=['Trimestre 1', 'Trimestre 2', 'Trimestre 3'],
#         right=True
#     )

#     # Ajouter les données filtrées à la liste
#     transformed_data.append(devoir_data)

# # Concaténer toutes les lignes dans un nouveau DataFrame
# notes_par_devoir = pd.concat(transformed_data, ignore_index=True)


In [11]:
import pandas as pd

# Initialiser une liste pour stocker les lignes du nouveau DataFrame
transformed_data = []

# Boucler sur chaque devoir (devoir1 à devoir24)
for i in range(1, 25):
    # Définir les noms de colonnes pour chaque devoir
    note_col = f'devoir{i}___note'
    coeff_col = f'devoir{i}___coeff'
    date_col = f'devoir{i}___date'
    periode_col = f'devoir{i}___periode_notation'

    # Filtrer les lignes où il y a une note pour le devoir i
    devoir_data = data_pronote['Notes'][['eleve_id', 'matiere', 'prof_id', note_col, coeff_col, date_col, periode_col]]

    # Renommer les colonnes pour un format générique
    devoir_data = devoir_data.rename(columns={
        note_col: 'note',
        coeff_col: 'coef',
        date_col: 'date',
        periode_col: 'trimestre'
    })

    # Supprimer les lignes sans note pour le devoir i
    devoir_data = devoir_data.dropna(subset=['note', 'coef', 'date', 'trimestre'])

    # Ajouter les données filtrées à la liste
    transformed_data.append(devoir_data)

# Concaténer toutes les lignes dans un nouveau DataFrame
notes_par_devoir = pd.concat(transformed_data, ignore_index=True)

# Afficher un aperçu du nouveau DataFrame
notes_par_devoir.head()


,eleve_id,matiere,prof_id,note,coef,date,trimestre
0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,ANGLAIS LVB,9SKAZVd_e50ujms6tkEJP9e_afvdckaaz3GN09KWnlU,"1,00","0,50",2024-01-29,Trimestre 2
1,Jmb2Zp7OzViHfStfxbPZejJQGwNhalrYDPUL9ackCNk,ANGLAIS LVB,9SKAZVd_e50ujms6tkEJP9e_afvdckaaz3GN09KWnlU,"8,50","0,50",2024-01-29,Trimestre 2
2,g34-PymcwTHigMQA-tcOWM7lTXmazA41KO7I5TfRLS4,ANGLAIS LVB,9SKAZVd_e50ujms6tkEJP9e_afvdckaaz3GN09KWnlU,"3,00","0,50",2024-01-29,Trimestre 2
3,L4QL8ltgetnUp0hCH1kvkkOXLhS4gL5w0theSjYVYc4,ANGLAIS LVB,9SKAZVd_e50ujms6tkEJP9e_afvdckaaz3GN09KWnlU,"8,00","0,50",2024-01-29,Trimestre 2
4,H_euoKbCaEqXVKQ3jtX7_Z9LaA4YN_Tj7-FHjOpAZqY,ANGLAIS LVB,9SKAZVd_e50ujms6tkEJP9e_afvdckaaz3GN09KWnlU,"3,00","0,50",2024-01-29,Trimestre 2


In [12]:
# Trier le DataFrame par la colonne 'date'
notes_par_devoir_sorted = notes_par_devoir.sort_values(by='note',ascending=False).reset_index(drop=True)

# Afficher un aperçu du DataFrame trié
notes_par_devoir_sorted['note'].unique()

array(['N.Rdu', 'N.Not', 'Inap', 'Disp', 'Abs*', 'Abs', '9,80', '9,75',
       '9,70', '9,60', '9,55', '9,50', '9,30', '9,25', '9,20', '9,10',
       '9,00', '88,00', '86,00', '85,00', '84,00', '82,50', '82,00',
       '81,00', '80,00', '8,80', '8,75', '8,70', '8,64', '8,50', '8,40',
       '8,30', '8,25', '8,18', '8,00', '79,00', '76,00', '74,50', '72,00',
       '71,50', '71,00', '70,50', '70,00', '7,80', '7,75', '7,70', '7,50',
       '7,25', '7,20', '7,00', '68,00', '66,00', '65,00', '64,50',
       '63,00', '62,00', '60,00', '6,82', '6,75', '6,50', '6,25', '6,00',
       '59,00', '56,50', '53,50', '51,00', '50,50', '50,00', '5,91',
       '5,75', '5,50', '5,40', '5,25', '5,20', '5,00', '49,50', '49,00',
       '48,00', '47,00', '46,00', '45,00', '44,75', '44,50', '44,00',
       '43,00', '42,00', '40,00', '4,80', '4,75', '4,50', '4,30', '4,25',
       '4,00', '39,50', '39,00', '38,50', '38,00', '37,50', '37,00',
       '36,50', '36,00', '35,50', '35,00', '34,00', '33,50', '32,50',

In [13]:
data_pronote['Notes'].head(3)

,matiere,classe/grpe,moy_eleve,devoir1___note,devoir2___note,devoir3___note,devoir4___note,devoir5___note,devoir6___note,devoir7___note,...,devoir17___periode_notation,devoir18___periode_notation,devoir19___periode_notation,devoir20___periode_notation,devoir21___periode_notation,devoir22___periode_notation,devoir23___periode_notation,devoir24___periode_notation,eleve_id,prof_id
0,ANGLAIS LVB,1A,"7,02","1,00","9,00","6,00","2,00","10,00","5,00","7,00",...,None,None,None,None,None,None,None,None,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,9SKAZVd_e50ujms6tkEJP9e_afvdckaaz3GN09KWnlU
1,ANGLAIS LVB,1A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,Wcv-LGZa8cjaMN8Qs1fgAw91nICobomNCcag577rDCI,9SKAZVd_e50ujms6tkEJP9e_afvdckaaz3GN09KWnlU
2,ANGLAIS LVB,1A,"19,16","8,50","18,00","20,00","19,50","19,00","19,00","20,00",...,None,None,None,None,None,None,None,None,Jmb2Zp7OzViHfStfxbPZejJQGwNhalrYDPUL9ackCNk,9SKAZVd_e50ujms6tkEJP9e_afvdckaaz3GN09KWnlU


In [14]:
import pandas as pd

# Fonction pour convertir les valeurs de note en flottant après avoir remplacé les virgules
def convert_to_float(column):
    return pd.to_numeric(column.astype(str).str.replace(',', '.'), errors='coerce')

# Extraire les colonnes de note et de date (devoir1 à devoir24)
note_columns = [f'devoir{i}___note' for i in range(1, 25)]
date_columns = [f'devoir{i}___date' for i in range(1, 25)]

# Appliquer la conversion en flottant pour toutes les colonnes de note
notes_df = data_pronote['Notes'][note_columns].apply(convert_to_float)

# Trouver la note la plus basse et la plus élevée
min_note = notes_df.min().min()  # Note la plus basse
max_note = notes_df.max().max()  # Note la plus élevée

# Trouver la date la plus proche et la plus lointaine
min_date = pd.to_datetime(data_pronote['Notes'][date_columns].stack(), errors='coerce').min()  # Date la plus proche
max_date = pd.to_datetime(data_pronote['Notes'][date_columns].stack(), errors='coerce').max()  # Date la plus lointaine

# Affichage des résultats
print("Note la plus basse :", min_note)
print("Note la plus élevée :", max_note)
print("Date la plus proche :", min_date)
print("Date la plus lointaine :", max_date)


Note la plus basse : 0.0
Note la plus élevée : 88.0
Date la plus proche : 2023-01-10 00:00:00
Date la plus lointaine : 2024-12-05 00:00:00


In [15]:
import os

# Chemin du dossier d'exportation
export_folder = "exported_data"
os.makedirs(export_folder, exist_ok=True)  # Crée le dossier s'il n'existe pas

# Fonction pour exporter les DataFrames dans le dossier spécifié
def export_dataframes(dataframes, folder_path):
    """
    Exporte chaque DataFrame dans un fichier CSV distinct dans le dossier spécifié.

    Parameters:
        dataframes (dict): Dictionnaire contenant les DataFrames à exporter.
        folder_path (str): Chemin du dossier d'exportation.
    """
    for name, df in dataframes.items():
        file_path = os.path.join(folder_path, f"{name}.csv")  # Chemin complet du fichier
        df.to_csv(file_path, index=False, sep=';')  # Export en CSV avec ';' comme séparateur
        print(f"DataFrame '{name}' exporté vers {file_path}")

# Ajout de tous les DataFrames à exporter, y compris ceux dans `data_pronote`
dataframes_to_export = {
    'notes_par_devoir': notes_par_devoir,                # DataFrame avec les notes par devoir
    **data_pronote  # Inclut tous les DataFrames contenus dans data_pronote
}

# Exporter tous les DataFrames
export_dataframes(dataframes_to_export, export_folder)


DataFrame 'notes_par_devoir' exporté vers exported_data/notes_par_devoir.csv
DataFrame 'Notesdevoir_t3' exporté vers exported_data/Notesdevoir_t3.csv
DataFrame 'Punition' exporté vers exported_data/Punition.csv
DataFrame 'Notesdevoir_t2' exporté vers exported_data/Notesdevoir_t2.csv
DataFrame 'Notesdevoir' exporté vers exported_data/Notesdevoir.csv
DataFrame 'Notesdevoir_t1' exporté vers exported_data/Notesdevoir_t1.csv
DataFrame 'Professeur' exporté vers exported_data/Professeur.csv
DataFrame 'Absenceseleves' exporté vers exported_data/Absenceseleves.csv
DataFrame 'Absencerepas' exporté vers exported_data/Absencerepas.csv
DataFrame 'Retards' exporté vers exported_data/Retards.csv
DataFrame 'Eleve' exporté vers exported_data/Eleve.csv
DataFrame 'Service' exporté vers exported_data/Service.csv
DataFrame 'Appreciationprofesseurs' exporté vers exported_data/Appreciationprofesseurs.csv
DataFrame 'Passagesinfirmerie' exporté vers exported_data/Passagesinfirmerie.csv
DataFrame 'Appreciationd